# **Proyek Predictive Analytics: Prediksi Penyakit Jantung**

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

## **1. Mengimpor Library yang Dibutuhkan**
Langkah pertama adalah mengimpor semua library Python yang akan digunakan.

In [1]:
# Untuk mengabaikan peringatan yang tidak relevan agar output lebih bersih
import warnings
warnings.filterwarnings('ignore')

# Library untuk data manipulation dan analysis
import pandas as pd
import numpy as np

# Library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns

# Library dari Scikit-learn untuk machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Mengatur agar plot ditampilkan inline di notebook
%matplotlib inline

## **2. Data Loading dan Understanding**
Pada tahap ini akan memuat dataset, memberinya nama kolom yang sesuai, dan melakukan eksplorasi data awal untuk memahami strukturnya.

In [ ]:
# URL dari dataset Heart Disease UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

# Nama kolom sesuai dengan dokumentasi dataset
column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
                'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

# Memuat dataset menggunakan pandas
df = pd.read_csv(url, header=None, names=column_names, na_values='?')

# Menampilkan 5 baris pertama dari dataframe
print("Data Awal (5 baris pertama):")
display(df.head())

# Mengubah nilai target: nilai > 0 berarti ada penyakit jantung (diubah menjadi 1)
df['target'] = (df['target'] > 0).astype(int)

## **3. Data Cleaning**
Pada tahap ini akan melakukan pembersihan data secara menyeluruh, meliputi:

1. Penanganan nilai yang hilang (missing values).
2. Penanganan data duplikat.
3. Penanganan outlier menggunakan metode IQR.

In [ ]:
# 1. Penanganan Missing Values
print("--- 1. Penanganan Missing Values ---")
print(f"Jumlah baris awal: {len(df)}")
print(f"Jumlah missing values sebelum dibersihkan:\n{df.isnull().sum()}\n")
df.dropna(inplace=True)
print(f"Jumlah baris setelah menghapus missing values: {len(df)}\n")

# 2. Penanganan Data Duplikat
print("--- 2. Penanganan Data Duplikat ---")
print(f"Jumlah data duplikat: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Jumlah data duplikat setelah dihapus: {df.duplicated().sum()}")
print(f"Jumlah baris setelah menghapus duplikat: {len(df)}\n")

# 3. Penanganan Outlier dengan Metode IQR
print("--- 3. Penanganan Outlier ---")
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# Filter data, membuang baris yang memiliki outlier di salah satu kolom numerik
# Kondisi `~` (not) digunakan untuk mempertahankan baris yang TIDAK outlier
df_clean = df[~((df[numerical_cols] < (Q1 - 1.5 * IQR)) | (df[numerical_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

print(f"Ukuran data sebelum menghapus outlier: {df.shape}")
print(f"Ukuran data setelah menghapus outlier: {df_clean.shape}")
print("Data telah bersih dan siap untuk analisis lebih lanjut.")

## **3. Exploratory Data Analysis (EDA)**
Melakukan analisis dan visualisasi pada data yang sudah bersih (df_clean) untuk mendapatkan wawasan.

In [ ]:
# Menampilkan informasi dasar dataframe
print("\nInformasi DataFrame:")
df.info()

In [ ]:
# Menampilkan statistik deskriptif dari data yang sudah bersih
print("Statistik Deskriptif (Data Bersih):")
display(df_clean.describe())

# Visualisasi distribusi variabel target pada data bersih
plt.figure(figsize=(6, 4))
sns.countplot(x='target', data=df_clean)
plt.title('Distribusi Variabel Target (Data Bersih)')
plt.xlabel('Status Penyakit Jantung (0: Sehat, 1: Sakit)')
plt.ylabel('Jumlah Pasien')
plt.show()

# Visualisasi korelasi antar fitur menggunakan heatmap pada data bersih
plt.figure(figsize=(12, 10))
sns.heatmap(df_clean.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap Korelasi (Data Bersih)')
plt.show()

## **4. Data Preparation**
Memisahkan fitur dan target, membagi data menjadi set pelatihan dan pengujian, serta melakukan standarisasi fitur.

In [ ]:
# Memisahkan fitur (X) dan target (y) dari dataframe bersih
X = df_clean.drop('target', axis=1)
y = df_clean['target']

# Membagi data menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # Menggunakan stratify=y untuk menjaga proporsi kelas target

print(f"Ukuran data latih: {X_train.shape}")
print(f"Ukuran data uji: {X_test.shape}\n")

# Standarisasi fitur numerik
scaler = StandardScaler()

# Melakukan fit dan transform pada data latih
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

# Hanya melakukan transform pada data uji (menggunakan scaler yang sudah di-fit)
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Data setelah standarisasi (5 baris pertama dari data latih):")
display(X_train.head())

## **5. Modeling**
Membangun, melatih, dan mengevaluasi dua model klasifikasi: Logistic Regression dan Random Forest.

In [ ]:
# Inisialisasi model
log_reg = LogisticRegression(random_state=42)
rand_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Melatih model Logistic Regression
print("Melatih model Logistic Regression...")
log_reg.fit(X_train, y_train)

# Melatih model Random Forest
print("Melatih model Random Forest...")
rand_forest.fit(X_train, y_train)

print("\nKedua model berhasil dilatih.")

## **6. Evaluation**
Evaluasi performa kedua model menggunakan berbagai metrik pada data uji.

In [ ]:
# Fungsi untuk mengevaluasi model dan menampilkan hasilnya
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"--- Evaluasi Model: {model_name} ---")
    print(f"Akurasi: {accuracy:.2f}")
    print(f"Presisi: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}\n")
    
    # Menampilkan classification report
    print("Laporan Klasifikasi:")
    print(classification_report(y_test, y_pred, target_names=['Sehat', 'Sakit Jantung']))
    
    # Menampilkan confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Sehat', 'Sakit Jantung'],
                yticklabels=['Sehat', 'Sakit Jantung'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Prediksi')
    plt.ylabel('Aktual')
    plt.show()
    
    return {'Akurasi': accuracy, 'Presisi': precision, 'Recall': recall, 'F1-Score': f1}

# Mengevaluasi kedua model
log_reg_metrics = evaluate_model(log_reg, X_test, y_test, "Logistic Regression")
rand_forest_metrics = evaluate_model(rand_forest, X_test, y_test, "Random Forest")

### **Perbandingan Hasil Evaluasi Model**

In [ ]:
# Membuat DataFrame untuk membandingkan metrik
metrics_df = pd.DataFrame({
    'Logistic Regression': log_reg_metrics,
    'Random Forest': rand_forest_metrics
}).T # .T untuk transpose agar model menjadi baris

print("Perbandingan Metrik Evaluasi:")
display(metrics_df.round(3))

# Kesimpulan
best_model_name = metrics_df['F1-Score'].idxmax()
print(f"\nKesimpulan: Model terbaik berdasarkan F1-Score adalah **{best_model_name}**.")